In [3]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date
from fredapi import Fred
#import chart_wizard2 as cw
import matplotlib.pyplot as plt
import openpyxl as ox
import math

In [23]:
### 주요 함수를 정의한다

def get_clean_data(df, year):
    
    con_1 = ~df['시가총액'].isnull()
    con_2 = ~df['자본총계'].isnull()
    con_3 = ~df['P/B(Adj., FY End)'].isnull()
    con_4 = ~df['ROA'].isnull()
    condition_1 = con_1 | con_2 | con_3 | con_4

    ## 우선주자본금 측정시 오류를 제거하기 위한 장치
    df.loc[:, '우선주자본금'] = df['우선주자본금'].fillna(0)

    con_1 = df['P/B(Adj., FY End)'] > 0
    con_2 = df['결산월'] == 12
    con_3 = df['상장일'] < str(year-1)+'-01-01'
    con_4 = df['자본총계'] > df['우선주자본금']

    condition_2 = con_1 & con_2 & con_3 & con_4

    df = df[condition_1 & condition_2]

    #ni=~pd.isnull(df['당기순이익']) 
    #roa=(~pd.isnull(df['ROA']) & ~pd.isnull(df['ROA.1']))
    #ocf = (~pd.isnull(df['영업활동으로인한현금흐름']) & ~pd.isnull(df['영업활동으로인한현금흐름.1']))
    #debt= (~pd.isnull(df['비유동부채']) & ~pd.isnull(df['비유동부채.1']))
    #asset= (~pd.isnull(df['자산총계']) & ~pd.isnull(df['자산총계.1']))
    #turn=(~pd.isnull(df['총자산회전율']) & ~pd.isnull(df['총자산회전율.1']))

    #condition_3= ni & roa & ocf & debt & asset & turn

    #df = df[condition_3]
    ## 수정주가가 누락된 기업들의 정보 제거
    #df = df.dropna(subset=df.filter(like='수정주가').columns)
        ##금융기업 제거
    finance = ['은행', '종금', '화재', '증권', '금융', '보험', '손보', '손해']

    for f in finance:

        df = df[~df['Name'].str.contains(f)]
        
    return df


def get_SMB_HML(fin_df, num1, num2):
    fin_df.loc[:, 'BE_ME'] = fin_df['시가총액']/(fin_df['자본총계'] - fin_df['우선주자본금'])
    fin_df.loc[:, 'SMB_grp'] = pd.qcut(fin_df['시가총액'], num1, labels=range(1,num1+1)) 
    new_series = fin_df.groupby(['SMB_grp'])['BE_ME'].apply(lambda x: pd.qcut(x, num2, labels=range(1, num2+1)))
    new_series.name = 'HML_grp'
    merged_fin_df = pd.concat([fin_df, new_series], axis=1)
    return merged_fin_df

In [87]:
#print(year) 
path= r'C:\Users\stox1\Desktop\research\New_Factor_Model\New+3_factor_조은비\data'
path2 = r'C:\Users\stox1\Desktop\research\New_Factor_Model\New_3_factor\전상장기업_코드리스트_since_2008_FF.xlsx'
year = 2008
df = pd.read_csv(path+'/'+str(year)+'_F.csv', index_col = 'Code')
df['상장일'] = pd.to_datetime(df['상장일'], format='%Y%m%d')
price_df = pd.read_excel(path2, skiprows=[0,1,2,3,4,5,6,8,9,10,11,12,13,14,15], index_col='Code')

In [86]:
# 수정주가가 포함된 컬럼 제거
col_list = []
for col in df.columns:
    if "수정주가" not in col:
        col_list.append(col)
print(col_list)
df = df[col_list]

## price_df에서 전체기간 동안의 로그리턴을 산출하여 df를 구성한다 
log_return_df = np.log(price_df/price_df.shift(1))

['Name', '결산월', '상장일', '60개월베타', '60개월로그베타', '시가총액', '시가총액.1', '자본총계', '자산총계', '자산총계.1', '우선주자본금', 'P/E(Adj., FY End)', 'P/B(Adj., FY End)', 'P/S(Adj., FY End)', 'EV/EBITDA', '매출액', '매출총이익', '영업이익', '당기순이익', 'ROA', 'ROA.1', 'ROE', 'ROIC', '영업활동으로인한현금흐름', '영업활동으로인한현금흐름.1', '비유동부채', '비유동부채.1', '자산총계.2', '자산총계.3', '유동비율', '유동비율.1', '매출총이익률', '매출총이익률.1', '총자산회전율', '총자산회전율.1']


In [84]:
company_dict ={}

for year in range(2008, 2009):
    df = pd.read_csv(path+'/'+str(year)+'_F.csv', index_col = 'Code')
    df['상장일'] = pd.to_datetime(df['상장일'], format='%Y%m%d')
    data_df = get_clean_data(df, year)
    merged_fin_df = get_SMB_HML(data_df, 2, 3)     ## size group수, HML group 수 
    group_df= merged_fin_df[['Name', 'SMB_grp', 'HML_grp']]
    company_dict[year] = group_df

In [43]:
small_list = group_df.loc[group_df['SMB_grp']==1].index.tolist()
small_price_df = price_df[small_list]
logreg_collist_1 = []

for col in small_list:
    small_price_df[col+'logret'] = np.log(small_price_df[col]) - np.log(small_price_df[col].shift(1))
    logreg_collist_1.append(col+'logret')
# logreg_collist_1
large_list = group_df.loc[group_df['SMB_grp']==2].index.tolist()
large_price_df = price_df[large_list]
logreg_collist_2 = []
for col in large_list:
    large_price_df[col+'logret'] = np.log(large_price_df[col]) - np.log(large_price_df[col].shift(1))
    logreg_collist_2.append(col+'logret')
# logreg_collist_2
large_rtn_df = large_price_df.filter(like='logret')
large_rtn_df['daily_mean_rtn'] =large_rtn_df.mean(axis=1)
small_rtn_df = small_price_df.filter(like='logret')
small_rtn_df['daily_mean_rtn'] =small_rtn_df.mean(axis=1)

high_BEME_list = group_df.loc[group_df['HML_grp']==1].index.tolist()
high_price_df = price_df[high_BEME_list]

low_BEME_list = group_df.loc[group_df['HML_grp']==3].index.tolist()
low_price_df = price_df[low_BEME_list]

highreg_collist = []
for col in high_BEME_list:
    high_price_df[col+'logret'] = np.log(high_price_df[col]) - np.log(high_price_df[col].shift(1))
    highreg_collist.append(col+'logret')
# highreg_collist

lowreg_collist = []
for col in low_BEME_list:
    low_price_df[col+'logret'] = np.log(low_price_df[col]) - np.log(low_price_df[col].shift(1))
    lowreg_collist.append(col+'logret')
# lowreg_collist
high_rtn_df = high_price_df.filter(like='logret')
high_rtn_df['high_BEME_daily_mean_rtn'] =high_rtn_df.mean(axis=1)

low_rtn_df = low_price_df.filter(like='logret')
low_rtn_df['low_BEME_daily_mean_rtn'] =low_rtn_df.mean(axis=1)

factor_df = pd.concat([small_rtn_df['daily_mean_rtn'], large_rtn_df['daily_mean_rtn'], high_rtn_df['high_BEME_daily_mean_rtn'], low_rtn_df['low_BEME_daily_mean_rtn']], axis=1)
factor_df.columns = ['small_mean_rtn', 'big_mean_rtn', 'high_BEME_daily_mean_rtn', 'low_BEME_daily_mean_rtn']

factor_df['SMB'] = factor_df['small_mean_rtn']- factor_df['big_mean_rtn']
factor_df['HML'] = factor_df['high_BEME_daily_mean_rtn']- factor_df['low_BEME_daily_mean_rtn']

C:\Users\stox1\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


['A000250logret',
 'A000420logret',
 'A000440logret',
 'A000590logret',
 'A000760logret',
 'A000890logret',
 'A001000logret',
 'A001070logret',
 'A001140logret',
 'A001190logret',
 'A001360logret',
 'A001420logret',
 'A001550logret',
 'A001560logret',
 'A001570logret',
 'A001770logret',
 'A001810logret',
 'A001820logret',
 'A002140logret',
 'A002200logret',
 'A002210logret',
 'A002220logret',
 'A002230logret',
 'A002290logret',
 'A002420logret',
 'A002450logret',
 'A002540logret',
 'A002600logret',
 'A002680logret',
 'A002700logret',
 'A002720logret',
 'A002760logret',
 'A002870logret',
 'A002880logret',
 'A003010logret',
 'A003060logret',
 'A003190logret',
 'A003310logret',
 'A003650logret',
 'A003660logret',
 'A003680logret',
 'A003720logret',
 'A004060logret',
 'A004100logret',
 'A004250logret',
 'A004270logret',
 'A004320logret',
 'A004410logret',
 'A004540logret',
 'A004590logret',
 'A004700logret',
 'A004740logret',
 'A004780logret',
 'A004820logret',
 'A004830logret',
 'A004840l